## 말라리아 셀 이미지
* 말라리아 스크리너 연구 활동의 분할된 세포의 얇은 혈액 도말 슬라이드 이미지
* 리소스가 제한된 지역에서 현미경 전문가의 부담을 줄이고 진단 정확도를 개선하기 위해 NLM(National Library of Medicine)의 일부인 Lister Hill National Center for Biomedical Communications(LHNCBC)의 연구원들은 모바일 애플리케이션을 개발
* 방글라데시 치타공 의과대학 병원에서 150명의 P. falciparum 감염자와 50명의 건강한 환자의 Giemsa 염색 얇은 혈액 도말 슬라이드를 수집하고 사진을 촬영
* 적혈구를 감지하고 분할하기 위해 레벨 세트 기반 알고리즘을 적용

### 이미지 출처 
* [LHNCBC Full Download List](https://lhncbc.nlm.nih.gov/LHC-downloads/downloads.html#malaria-datasets)
* [Malaria Cell Images Dataset | Kaggle](https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria)
* 관련 논문 : [Performance evaluation of deep neural ensembles toward malaria parasite detection in thin-blood smear images [PeerJ]](https://peerj.com/articles/6977/)
* 해당 논문의 github : [sivaramakrishnan-rajaraman/Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images: This study evaluates the performance of custom and pretrained CNNs and construct an optimal model ensemble toward the challenge of classifying parasitized and normal cells in thin blood smear images. The results obtained are encouraging and superior to the state-of-the-art.](https://github.com/sivaramakrishnan-rajaraman/Deep-Neural-Ensembles-toward-Malaria-Parasite-Detection-in-Thin-Blood-Smear-Images)

<img src="https://i.imgur.com/okCbMzc.png" width="400">

## 라이브러리 로드

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

## 이미지 폴더 보기

In [ ]:
# 이미지 다운로드
# !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip

In [ ]:
# images 폴더 만들기
# !mkdir images

In [ ]:
# images 폴더에 다운로드 받은 파일 압축 해제하기
# !unzip cell_images.zip -d images 

In [ ]:
import os
for root, dirs, files in os.walk("./images/"):
    print(root, dirs, len(files))

## 일부 이미지 미리보기

In [ ]:
import cv2

upic = 'images/Uninfected/C100P61ThinF_IMG_20150918_144104_cell_131.png'
apic = 'images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_164.png'
plt.figure(1, figsize = (15 , 7))
plt.subplot(1 , 2 , 1)
print(cv2.imread(upic).shape)
plt.imshow(cv2.imread(upic))
plt.title('Uninfected Cell')
plt.xticks([]) , plt.yticks([])

plt.subplot(1 , 2 , 2)
plt.imshow(cv2.imread(apic))
print(cv2.imread(apic).shape)
plt.title('Infected Cell')
plt.xticks([]) , plt.yticks([])

plt.show()

## 이미지 사이즈 설정
* 이미지의 사이즈가 불규칙하면 학습을 할 수 없기 때문에 리사이즈할 크기를 지정합니다.

In [ ]:
width = 128
height = 128

## 데이터셋 나누기
* 학습, 검증 세트를 나눕니다.

In [ ]:
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2)

## 학습 세트

In [ ]:
trainDatagen = datagen.flow_from_directory(directory = 'images/cell_images/',
                                           target_size = (width,height),
                                           class_mode = 'binary',
                                           batch_size = 64,
                                           subset='training')

## 검증 세트

In [ ]:
valDatagen = datagen.flow_from_directory(directory = 'images/',
                                         target_size =(width,height),
                                         class_mode = 'binary',
                                         batch_size = 64,
                                         subset='validation')

## 레이어 설정

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3),activation='relu', input_shape=(128,128,3)))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(filters=64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

## 모델 요약

In [ ]:
model.summary()

## 컴파일

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

## 학습

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
history = model.fit_generator(generator = trainDatagen,
                              steps_per_epoch = len(trainDatagen),
                              epochs = 10,
                              validation_data = valDatagen,
                              validation_steps=len(valDatagen),
                              callbacks=[early_stop])